In [39]:
import json
import pandas as pd
import numpy as np
import os
import time

In [40]:
def load_and_prepare_data(input_json_path):
    """
    Читает JSON конфиг и единый CSV файл с колонками:
    user_id, product_id, lama_proba.
    
    Возвращает:
    - clients: список user_id
    - product_revenues: словарь product_id → доход
    - channels_data: словарь каналов с cost, prob
    - channel_limits: лимиты каналов
    - df_affinity: очищенный DataFrame
    - budget: бюджет кампании
    """
    # ----------------------
    # 1. Читаем конфиг
    # ----------------------
    with open(input_json_path, 'r', encoding='utf-8') as f:
        config = json.load(f)

    budget = float(config["budget"])
    channels_config = config["channels"]
    product_revenues = config["products"]

    # ----------------------
    # 2. Подготавливаем каналы
    # ----------------------
    channels_data = {}
    channel_limits = {}

    for name, params in channels_config.items():
        limit, cost, prob = params
        channels_data[name] = {
            "cost": float(cost),
            "prob": float(prob)
        }
        channel_limits[name] = int(limit)

    # ----------------------
    # 3. Читаем единый CSV
    # ----------------------
    df = pd.read_csv("test.csv")

    expected_columns = ["user_id", "product_id", "lama_proba"]
    for col in expected_columns:
        if col not in df.columns:
            raise ValueError(f"В CSV нет обязательной колонки: {col}")

    df = df.copy()

    # Очищаем
    df["lama_proba"] = pd.to_numeric(df["lama_proba"], errors="coerce")
    df = df.dropna(subset=["lama_proba"])
    df = df[df["lama_proba"] > 0]

    df["affinity_prob"] = df["lama_proba"]


    # Проверяем product_id
    df["product_id"] = df["product_id"].astype(str)

    # Фильтруем только продукты, которые есть в JSON
    df = df[df["product_id"].isin(product_revenues.keys())]
    df["product_revenue"] = df["product_id"].map(product_revenues)


    if df.empty:
        raise ValueError("CSV не содержит совпадающих product_id с JSON или вероятности равны нулю.")

    # ----------------------
    # 4. Список клиентов
    # ----------------------
    clients = df["user_id"].unique().tolist()

    return clients, product_revenues, channels_data, channel_limits, df, budget


In [41]:
def solve_fast_heuristic(df_base, channels_data, channel_limits, budget):
    """
    Быстрый жадный алгоритм на Pandas/Numpy.
    Сортирует предложения по ROI и выбирает лучшие с учетом ограничений.
    """
    print(f"Начинаем оптимизацию для {len(df_base)} пар клиент-продукт...")
    start_time = time.time()
    
    # 1. Разворачиваем данные по каналам
    channel_records = []
    for c_name, c_data in channels_data.items():
        channel_records.append({
            'canal_id': c_name,
            'channel_cost': c_data['cost'],
            'channel_prob': c_data['prob']
        })
    
    df_channels = pd.DataFrame(channel_records)
    
    # Декартово произведение (Client, Product) x (Channel)
    df_full = df_base.merge(df_channels, how='cross')
    
    # 2. Расчет метрик
    df_full['expected_revenue'] = (
        df_full['product_revenue'] * 
        df_full['affinity_prob'] * 
        df_full['channel_prob']
    )
    
    # Фильтруем убыточные
    df_full = df_full[df_full['expected_revenue'] > df_full['channel_cost']].copy()
    
    # ROI Score
    df_full['roi_score'] = df_full['expected_revenue'] / df_full['channel_cost']
    
    # 3. Сортировка
    print("Сортировка вариантов...")
    df_full.sort_values(by='roi_score', ascending=False, inplace=True)
    
    # 4. Жадный выбор
    used_clients = set()
    current_channel_usage = {ch: 0 for ch in channels_data}
    current_budget_spent = 0.0
    
    selected_indices = []
    
    # Numpy arrays for speed
    data_client = df_full['user_id'].values
    data_channel = df_full['canal_id'].values
    data_cost = df_full['channel_cost'].values
    data_indices = df_full.index.values 
    
    limits_lookup = {k: channel_limits.get(k, float('inf')) for k in channels_data}
    
    print("Выбор лучших предложений...")
    
    for i in range(len(data_client)):
        cost = data_cost[i]
        
        if current_budget_spent + cost > budget:
            continue 
            
        client = data_client[i]
        
        if client in used_clients:
            continue
            
        channel = data_channel[i]
        
        if current_channel_usage[channel] >= limits_lookup[channel]:
            continue
            
        # Success
        selected_indices.append(data_indices[i])
        used_clients.add(client)
        current_channel_usage[channel] += 1
        current_budget_spent += cost
        
        if budget - current_budget_spent < 0.1:
            break

    # Результат
    df_results = df_full.loc[selected_indices].copy()
    df_results.rename(columns={'channel_cost': 'cost'}, inplace=True)
    
    print(f"Оптимизация завершена за {time.time() - start_time:.2f} сек.")
    print(f"Потрачено: {current_budget_spent:.2f} из {budget}")
    
    return df_results


In [42]:
def generate_reports(df_results, budget, product_revenues, channels_data, channel_limits, output_json_path, output_csv_path):
    """
    Формирует и сохраняет отчеты в CSV и JSON.
    """
    # CSV
    out_columns = ['product_id', 'user_id', 'canal_id']
    if not df_results.empty:
        df_results[out_columns].to_csv(output_csv_path, index=False)
    else:
        pd.DataFrame(columns=out_columns).to_csv(output_csv_path, index=False)
        
    # JSON
    if df_results.empty:
        report = {
            "summary": [float(budget), 0.0, 0.0, 0.0, 0.0, 0],
            "channels_usage": {},
            "products_distribution": {}
        }
    else:
        actual_spend = float(df_results['cost'].sum())
        expected_revenue = float(df_results['expected_revenue'].sum())
        actual_spend_percent = round((actual_spend / budget) * 100, 1)
        expected_roi = ((expected_revenue - actual_spend) / actual_spend * 100) if actual_spend > 0 else 0.0
        reach_clients = int(len(df_results))
        
        summary_vec = [
            float(budget),
            round(actual_spend, 2),
            actual_spend_percent,
            round(expected_revenue, 2),
            round(expected_roi, 1),
            reach_clients
        ]
        
        channels_usage = {}
        # Агрегация по каналам из датафрейма для скорости
        if not df_results.empty:
            ch_stats = df_results.groupby('canal_id').agg(
                count=('user_id', 'count'),
                cost=('cost', 'sum'),
                rev=('expected_revenue', 'sum')
            )
        
        for ch in channels_data.keys():
            if not df_results.empty and ch in ch_stats.index:
                row = ch_stats.loc[ch]
                channels_usage[ch] = [
                    int(row['count']),
                    float(round(row['cost'], 2)),
                    float(round(row['rev'], 2))
                ]
            else:
                channels_usage[ch] = [0, 0.0, 0.0]

        products_distribution = {}
        # Агрегация по продуктам
        if not df_results.empty:
            prod_stats = df_results.groupby('product_id').agg(
                count=('user_id', 'count'),
                avg_rev=('expected_revenue', 'mean')
            )

        for prod in product_revenues.keys():
            if not df_results.empty and prod in prod_stats.index:
                row = prod_stats.loc[prod]
                products_distribution[prod] = [
                    int(row['count']),
                    float(round(row['avg_rev'], 2))
                ]
            else:
                products_distribution[prod] = [0, 0.0]
            
        report = {
            "summary": summary_vec,
            "channels_usage": channels_usage,
            "products_distribution": products_distribution
        }

    with open(output_json_path, 'w', encoding='utf-8') as f:
        json.dump(report, f, indent=4, ensure_ascii=False)
        
    return output_json_path

In [43]:
def optimize_marketing_campaign(input_json_path):
    output_json = 'report.json'
    output_csv = 'results.csv'
    
    try:
        # 1. Загрузка
        (
    clients,
    prod_revs,
    chan_data,
    chan_limits,
    df_base,
    budget
) = load_and_prepare_data(input_json_path)

        
        # 2. Оптимизация
        df_results = solve_fast_heuristic(
            df_base, 
            chan_data, 
            chan_limits, 
            budget
        )
        
        # 3. Отчет
        report_file = generate_reports(
            df_results, 
            budget, 
            prod_revs, 
            chan_data, 
            chan_limits, 
            output_json, 
            output_csv
        )
        
        return report_file
        
    except Exception as e:
        print(f"Критическая ошибка: {e}")
        import traceback
        traceback.print_exc()
        return None

In [53]:
# 2. Запускаем функцию
result_file = optimize_marketing_campaign('input.json')

# 3. Читаем и выводим результат для проверки
if result_file:
    print("\n--- Содержимое отчета (JSON) ---")
    with open(result_file, 'r') as f:
        print(json.dumps(json.load(f), indent=2))
        
    print("\n--- Пример CSV (первые 5 строк) ---")
    try:
        print(pd.read_csv('results.csv').head())
    except:
        print("Файл results.csv пуст или не создан.")

Начинаем оптимизацию для 179194 пар клиент-продукт...
Сортировка вариантов...
Выбор лучших предложений...
Оптимизация завершена за 0.35 сек.
Потрачено: 459.07 из 341386.0

--- Содержимое отчета (JSON) ---
{
  "summary": [
    341386.0,
    459.07,
    0.1,
    431406.63,
    93873.2,
    52290
  ],
  "channels_usage": {
    "sms": [
      0,
      0.0,
      0.0
    ],
    "email": [
      45768,
      401.83,
      430208.77
    ],
    "push": [
      6522,
      57.24,
      1197.86
    ],
    "phone": [
      0,
      0.0,
      0.0
    ]
  },
  "products_distribution": {
    "credit_card": [
      7707,
      6.35
    ],
    "debit_card": [
      3263,
      0.2
    ],
    "loan": [
      22534,
      12.37
    ],
    "auto_c": [
      12802,
      7.21
    ],
    "subscr": [
      1042,
      0.02
    ],
    "cash": [
      4942,
      2.15
    ]
  }
}

--- Пример CSV (первые 5 строк) ---
  product_id  user_id canal_id
0       loan  5265653    email
1       loan  7072552    email
